# Вебинар 5. Домашнее задание

Само домашнее задание нахожится в конце ноутбука

---

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items

In [134]:
data = pd.read_csv('../retail_train/retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [103]:
item_features = pd.read_csv('../retail_train/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [104]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [105]:
n_items_before = data_train['item_id'].nunique()

data_train = prefilter_items(data_train)

n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 86865 to 73918


In [106]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

user_item_matrix.head(3)

item_id,25671,26093,26190,26426,26601,26636,26691,26738,26889,26941,...,17328345,17328742,17329473,17329749,17330255,17330511,17382205,17383227,17827644,17829232
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [107]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [108]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [109]:
%%time

model = AlternatingLeastSquares(factors=20, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


Wall time: 2.51 s


-----

# Домашнее задание

1. Изучите структуру модуля src
2. Перенесите функции prefilter_items и postfilter_items в модуль src.utils.py
3. Реализуйте функции get_similar_items_recommendation, get_similar_users_recommendation (они разбирались на вебинаре) и переместите в src.utils.py.  
postfilter можно пока оставить пустой (или как-то бороться с товаром 999999) - на Ваше усмотрение.  
В prefilter нужно реализовать закомментированные ограничения.
4. Создайте модуль src.recommenders.py. Напишите код для класса ниже и положите его в src.recommenders.py  
get_similar_items_recommendation и get_similar_users_recommendation - реализуйте их, как делалось на вебинаре
5. Проверьте, что все модули корректно импортируются
6. Если вы еще не прочитали [статью](https://habr.com/ru/company/hh/blog/347276/) о рекомендательных системах и поиске в hh.ru, то обязательно прочитайте

**Внимание!** модуль для вашего удобства - дополняйте его теми функциями, которые Вам пригодятся.  
Формальное задание только на prefiletr_items 

In [110]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [111]:
data_train.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price
0,2375,26984851472,1,999998,1,1.39,364,-0.60,1631,1,0.0,0.0,1.39
1,2375,26984851472,1,999996,1,0.82,364,0.00,1631,1,0.0,0.0,0.82
2,2375,26984851472,1,999996,1,0.99,364,-0.30,1631,1,0.0,0.0,0.99
3,2375,26984851472,1,999998,1,1.21,364,0.00,1631,1,0.0,0.0,1.21
4,2375,26984851472,1,999998,1,1.50,364,-0.39,1631,1,0.0,0.0,1.50


In [112]:
data_train['price'] = data_train['sales_value'] / data_train['quantity']
data_train.loc[data_train['quantity']==0, ['price']]=data_train['sales_value']
data_train.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price
0,2375,26984851472,1,999998,1,1.39,364,-0.60,1631,1,0.0,0.0,1.39
1,2375,26984851472,1,999996,1,0.82,364,0.00,1631,1,0.0,0.0,0.82
2,2375,26984851472,1,999996,1,0.99,364,-0.30,1631,1,0.0,0.0,0.99
3,2375,26984851472,1,999998,1,1.21,364,0.00,1631,1,0.0,0.0,1.21
4,2375,26984851472,1,999998,1,1.50,364,-0.39,1631,1,0.0,0.0,1.50


In [113]:
data_train.describe().T

,count,mean,std,min,25%,50%,75%,max
user_id,2278490.0,1.271764e+03,7.269816e+02,1.000000e+00,6.540000e+02,1.271000e+03,1.914000e+03,2.500000e+03
basket_id,2278490.0,3.294526e+10,3.964679e+09,2.698485e+10,3.003546e+10,3.214976e+10,3.433825e+10,4.129777e+10
day,2278490.0,3.491402e+02,1.676271e+02,1.000000e+00,2.080000e+02,3.510000e+02,4.940000e+02,6.350000e+02
item_id,2278490.0,2.007199e+06,3.009298e+06,2.567100e+04,9.999960e+05,9.999980e+05,9.999980e+05,1.782923e+07
quantity,2278490.0,1.006171e+02,1.153002e+03,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,8.963800e+04
sales_value,2278490.0,3.095110e+00,4.196106e+00,0.000000e+00,1.270000e+00,2.000000e+00,3.490000e+00,8.400000e+02
store_id,2278490.0,2.992061e+03,8.693638e+03,1.000000e+00,3.300000e+02,3.700000e+02,4.220000e+02,3.428000e+04
retail_disc,2278490.0,-5.393603e-01,1.236080e+00,-1.300200e+02,-6.900000e-01,-2.000000e-02,0.000000e+00,3.990000e+00
trans_time,2278490.0,1.562467e+03,4.025741e+02,0.000000e+00,1.306000e+03,1.615000e+03,1.846000e+03,2.359000e+03
week_no,2278490.0,5.056328e+01,2.394798e+01,1.000000e+00,3.000000e+01,5.100000e+01,7.100000e+01,9.100000e+01


In [114]:
data_train.trans_time.unique().shape

(1440,)

In [115]:
item_features.department.unique()

array(['GROCERY', 'MISC. TRANS.', 'PASTRY', 'DRUG GM', 'MEAT-PCKGD',
       'SEAFOOD-PCKGD', 'PRODUCE', 'NUTRITION', 'DELI', 'COSMETICS',
       'MEAT', 'FLORAL', 'TRAVEL & LEISUR', 'SEAFOOD', 'MISC SALES TRAN',
       'SALAD BAR', 'KIOSK-GAS', 'ELECT &PLUMBING', 'GRO BAKERY',
       'GM MERCH EXP', 'FROZEN GROCERY', 'COUP/STR & MFG', 'SPIRITS',
       'GARDEN CENTER', 'TOYS', 'CHARITABLE CONT', 'RESTAURANT', 'RX',
       'PROD-WHS SALES', 'MEAT-WHSE', 'DAIRY DELI', 'CHEF SHOPPE', 'HBC',
       'DELI/SNACK BAR', 'PORK', 'AUTOMOTIVE', 'VIDEO RENTAL', ' ',
       'CNTRL/STORE SUP', 'HOUSEWARES', 'POSTAL CENTER', 'PHOTO', 'VIDEO',
       'PHARMACY SUPPLY'], dtype=object)

In [116]:
drop_popular_amount=5000

popularity_table = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity_table.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity_table.sort_values('n_sold', ascending=False, inplace=True)

top_list = popularity_table.head(drop_popular_amount).item_id.tolist()

bot_list = popularity_table.tail(drop_popular_amount).item_id.tolist()

bot_list[:5]

[1030060, 10254864, 10254843, 10254820, 1030072]

In [117]:
uninteresting_department_list = ['GROCERY', 'MISC. TRANS.']
uninteresting_list = item_features.loc[item_features['department'].isin(uninteresting_department_list), :]['item_id'].unique().tolist()
uninteresting_list[:5]

[25671, 26081, 26190, 26355, 26426]

In [135]:
def prefilter_items(data_train, 
                    item_features=None,
                    drop_popular_amount=100, 
                    drop_unpopular_amount=100, 
                    uninteresting_department_list=None,
                    low_price=1,
                    high_price=100):
    '''
    # Оставим только 5000 самых популярных товаров
    popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
    popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
    top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()
    #добавим, чтобы не потерять юзеров
    data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999 
    '''    
    popularity_table = data_train.groupby('item_id')['quantity'].sum().reset_index()
    popularity_table.rename(columns={'quantity': 'n_sold'}, inplace=True)
    popularity_table.sort_values('n_sold', ascending=True, inplace=True)
    
    # Уберем самые популярные 
    top_list = popularity_table.head(drop_popular_amount).item_id.tolist()
    data_train.loc[data_train['item_id'].isin(top_list), 'item_id'] = 999999    
    # Уберем самые непопулряные 
    bot_list = popularity_table.tail(drop_unpopular_amount).item_id.tolist()
    data_train.loc[data_train['item_id'].isin(bot_list), 'item_id'] = 999998
    # Уберем товары, которые не продавались за последние 12 месяцев
    # Нет инфы. Времятранспортировки это я так понял не оно
    # Уберем не интересные для рекоммендаций категории (department)
    if uninteresting_department_list and item_features:
        uninteresting_list = uninteresting_list = item_features.loc[item_features['department'].\
                                                                    isin(uninteresting_department_list), :]\
                                                                    ['item_id'].unique().tolist()
        data_train.loc[data_train['item_id'].isin(uninteresting_list), 'item_id'] = 999997
    # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб. 
    data_train['price'] = data_train['sales_value'] / data_train['quantity']
    data_train.loc[data_train['quantity']==0, ['price']]=data_train['sales_value']
    data_train.loc[data_train['price'] < low_price, 'item_id'] = 999996
    # Уберем слишком дорогие товары
    data_train.loc[data_train['price'] > high_price, 'item_id'] = 999995

    return data_train

def postfilter_items():
    pass

In [146]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight


class MainRecommender:
    """Рекоммендации, которые можно получить из ALS
    
    Input
    -----
    user_item_matrix: pd.DataFrame
        Матрица взаимодействий user-item
    """
    
    def __init__(self, data, weighting=True):
        
        # your_code. Это не обязательная часть. Но если вам удобно что-либо посчитать тут - можно это сделать
        self.data = data.copy()
        self.user_item_matrix = self.prepare_matrix(data)  # pd.DataFrame
        self.id_to_itemid, self.id_to_userid,\
            self.itemid_to_id, self.userid_to_id = self.prepare_dicts(self.user_item_matrix)
        
        if weighting:
            self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T 
        
        self.model = self.fit(self.user_item_matrix)
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)
     
    @staticmethod
    def prepare_matrix(data):
        
        # your_code
        user_item_matrix = pd.pivot_table(data=data, 
                                          index='user_id', columns='item_id', 
                                          values='quantity', # Можно пробовать другие варианты
                                          aggfunc='count', 
                                          fill_value=0)
        
        user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit
        
        return user_item_matrix
    
    @staticmethod
    def prepare_dicts(user_item_matrix):
        """Подготавливает вспомогательные словари"""
        
        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))
        
        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id
     
    @staticmethod
    def fit_own_recommender(user_item_matrix):
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""
    
        own_recommender = ItemItemRecommender(K=1, num_threads=4)
        own_recommender.fit(csr_matrix(user_item_matrix).T.tocsr())
        
        return own_recommender
    
    @staticmethod
    def fit(user_item_matrix, n_factors=20, regularization=0.001, iterations=15, num_threads=4):
        """Обучает ALS"""
        
        model = AlternatingLeastSquares(factors=n_factors, 
                                             regularization=regularization,
                                             iterations=iterations,  
                                             num_threads=num_threads)
        model.fit(csr_matrix(user_item_matrix).T.tocsr())
        
        return model

    def get_similar_items_recommendation(self, user, N=5):
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""
        
        # your_code
        # Практически полностью реализовали на прошлом вебинаре
        
        def get_rec(model, x):
            recs = model.similar_items(self.itemid_to_id[x], N=2)
            top_rec = recs[1][0]
            return self.id_to_itemid[top_rec]
        
        data = self.data.loc[self.data['user_id']==self.userid_to_id[user], :]
        popularity = data.groupby(['item_id'])['quantity'].count().reset_index()
        popularity.sort_values('quantity', ascending=False, inplace=True)
        popularity['similar_recommendation'] = popularity['item_id'].apply(lambda x: get_rec(self.model, x))

        res = popularity['similar_recommendation'].unique()[:N]        
        
        #assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res
    
    def get_similar_users_recommendation(self, user, N=5):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""

        # your_code
        sparse_user_item=csr_matrix(self.user_item_matrix).T.tocsr()
        
        res = [self.id_to_itemid[rec[0]] for rec in 
                    self.model.recommend(userid=self.userid_to_id[user], 
                                         user_items=sparse_user_item,   # на вход user-item matrix
                                         N=N, 
                                         filter_already_liked_items=False, 
                                         #filter_items=[self.itemid_to_id[999999]], # !!! 
                                         recalculate_user=True)]

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res

----

In [147]:
import warnings
warnings.filterwarnings('ignore')
data_train_filter = prefilter_items(data_train)
data_test_filter = prefilter_items(data_test)

In [148]:
recomendation = MainRecommender(data_train_filter)

In [149]:
recomendation.get_similar_items_recommendation(2375)

array([  999998, 13776705,  8018773,  1399338,  1155121], dtype=int64)

In [158]:
recomendation.get_similar_users_recommendation(2375)

[999999, 34873, 34160, 269894, 32916]

Проверка, что все работает

In [ ]:
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender
_recomendation_ = MainRecommender(data_train_filter)
print(_recomendation_.get_similar_items_recommendation(2375))
print()
print(_recomendation_.get_similar_users_recommendation(2375))

In [152]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

In [153]:
%%time
result['items_recommendation'] = result['user_id'].apply(lambda x: recomendation.get_similar_items_recommendation(x))
result['users_recommendation'] = result['user_id'].apply(lambda x: recomendation.get_similar_users_recommendation(x))

Wall time: 9min 11s


In [154]:
result.apply(lambda row: precision_at_k(row['items_recommendation'], row['actual']), axis=1).mean()

0.36990037563284917

In [155]:
result.apply(lambda row: precision_at_k(row['users_recommendation'], row['actual']), axis=1).mean()

9.794319294809012e-05

In [156]:
result.head(10)

,user_id,actual,items_recommendation,users_recommendation
0,1,"[999996, 856942, 999998, 889248, 943316, 10608...",[],"[32916, 34873, 40885, 34160, 417578]"
1,3,"[835476, 999996, 999998, 958154]","[999996, 999998, 819400, 906675, 905383]","[34873, 34160, 999999, 269894, 43020]"
2,6,"[999996, 999998, 1104227, 14106553, 909479, 65...","[999996, 999998, 917857, 824252, 5995589]","[32916, 999999, 40885, 96755, 417578]"
3,7,"[999998, 999996, 993838, 1056418, 1108844, 111...","[999996, 999998, 892473, 973238, 6979320]","[999999, 34873, 34160, 66398, 28272]"
4,8,"[999998, 992977, 999996, 9337369, 15629920, 81...","[999996, 999998, 909611, 13776705, 7147475]","[1124560, 1033814, 1063899, 12487147, 8019602]"
5,9,"[999998, 889692]","[999996, 999998, 862314, 1100830, 825717]","[34873, 34160, 999999, 269894, 43020]"
6,13,"[999996, 829197, 999998, 1100973, 7409999, 122...","[999996, 999998, 920967, 853197, 929893]","[34873, 34160, 999999, 269894, 43020]"
7,14,"[999998, 965693, 999996, 925205, 954922, 6979594]","[999996, 999998, 13877036, 991303, 13006879]","[999999, 34873, 244960, 32916, 41044]"
8,15,"[999998, 999996]","[999996, 999998, 1042380, 10182698, 964868]","[32916, 40885, 34873, 244960, 999999]"
9,16,"[1062973, 999996, 13007710]","[999996, 999998, 1104058, 12757596, 13955724]","[1124560, 1033814, 1063899, 12487147, 8019602]"
